<a href="https://colab.research.google.com/github/elichen/aoc2018/blob/main/Day_22_Mode_Maze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
def create_cave_system_int(depth, target):
    x, y = target
    erosion_level = np.zeros((x + 1, y + 1), dtype=int)
    type_array = np.zeros((x + 1, y + 1), dtype=int)

    for i in range(x + 1):
        for j in range(y + 1):
            if (i, j) in [(0, 0), target]:
                geo_idx = 0
            elif j == 0:
                geo_idx = i * 16807
            elif i == 0:
                geo_idx = j * 48271
            else:
                geo_idx = erosion_level[i - 1, j] * erosion_level[i, j - 1]

            erosion = (geo_idx + depth) % 20183
            erosion_level[i, j] = erosion
            type_array[i, j] = erosion % 3

    return type_array

# Test the function with integers for types
target = (5,746)
depth = 4002
m = create_cave_system_int(depth, target)
np.sum(m)

4479

In [16]:
def create_cave_system_int2(depth, target_x, target_y, padding = 100):
    x, y = target_x+100, target_y+100
    erosion_level = np.zeros((x + 1, y + 1), dtype=int)
    type_array = np.zeros((x + 1, y + 1), dtype=int)

    for i in range(x + 1):
        for j in range(y + 1):
            if (i, j) in [(0, 0), target]:
                geo_idx = 0
            elif j == 0:
                geo_idx = i * 16807
            elif i == 0:
                geo_idx = j * 48271
            else:
                geo_idx = erosion_level[i - 1, j] * erosion_level[i, j - 1]

            erosion = (geo_idx + depth) % 20183
            erosion_level[i, j] = erosion
            type_array[i, j] = erosion % 3

    return type_array

# Test the function with integers for types
m = create_cave_system_int2(depth, target[0], target[1], padding=10)
m

array([[0, 0, 0, ..., 0, 1, 1],
       [2, 0, 0, ..., 0, 2, 1],
       [0, 0, 2, ..., 2, 0, 2],
       ...,
       [2, 0, 2, ..., 1, 0, 2],
       [1, 0, 2, ..., 1, 2, 1],
       [0, 2, 2, ..., 2, 1, 2]])

In [17]:
import heapq

def dijkstra_shortest_path(type_array, target):
    # Define the tools and region types
    ROCKY, WET, NARROW = 0, 1, 2
    TORCH, CLIMBING, NEITHER = 0, 1, 2

    # Tools valid for each region type
    valid_tools = {
        ROCKY: {TORCH, CLIMBING},
        WET: {CLIMBING, NEITHER},
        NARROW: {TORCH, NEITHER}
    }

    # Starting point
    start = (0, 0, TORCH)

    # Priority queue for Dijkstra (time, state)
    pq = []
    heapq.heappush(pq, (0, start))

    # Visited states with the shortest time recorded
    visited = {start: 0}

    while pq:
        time, (x, y, tool) = heapq.heappop(pq)

        # Check if target is reached with torch
        if (x, y) == target and tool == TORCH:
            return time

        # Explore adjacent cells
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < type_array.shape[0] and 0 <= ny < type_array.shape[1]:
                next_region_type = type_array[nx, ny]
                if tool in valid_tools[next_region_type]:
                    next_state = (nx, ny, tool)
                    new_time = time + 1
                    if next_state not in visited or new_time < visited[next_state]:
                        heapq.heappush(pq, (new_time, next_state))
                        visited[next_state] = new_time

        # Explore tool change
        current_region_type = type_array[x, y]
        for next_tool in valid_tools[current_region_type]:
            if next_tool != tool:
                next_state = (x, y, next_tool)
                new_time = time + 7
                if next_state not in visited or new_time < visited[next_state]:
                    heapq.heappush(pq, (new_time, next_state))
                    visited[next_state] = new_time

    return -1  # If target is not reachable

# Test the Dijkstra's algorithm implementation
shortest_time_dijkstra = dijkstra_shortest_path(m, target)
shortest_time_dijkstra

1032